In [359]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time

import pandas as pd

driver = webdriver.Chrome()

In [360]:
def load_page():
    # 페이지가 로딩될 때까지 기다림 (예: 10초)
    wait = WebDriverWait(driver, 3)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    time.sleep(0.3)
    
    # 페이지 소스 가져오기
    html_content = driver.page_source

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html_content, "html.parser")

    
    return soup
    
def check_valid(soup):
    time.sleep(0.1)
    valid = soup.select_one('#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.data_area > div.btn_wrap > div > a > span > b')

    # 카드 신청 가능할 경우, '카드 신청'
    # 카드 신청 불가능할 경우, '신규발급이 중단된 카드입니다.'
#     try:
#         if valid.get_text() =='신규발급이 중단된 카드입니다.':
#             valid_con=False
#         else:
#             valid_con=True
#     except:
#         valid_con = False
        
    # 아무것도 없는 경우(카드 가입, 신규가입 불가 등 아무런 문구 없이 카드만 나오는 경우)
    if valid:
        valid_con=False
    else:
        valid_con=True

    return valid_con

def get_card_info(i):
    # 해당 위치의 모든 dl 태그 가져오기
    items = driver.find_elements(By.XPATH, '//*[@id="q-app"]/section/div[1]/section/div/article[2]/div[1]/dl')

    # 각 dl 태그에 대해 반복적으로 작업 수행
    for item in items:
        # 스크롤을 맨 아래까지 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.1)
        
        # 아이템 클릭
        item.click()
        
    # 페이지 소스 가져오기
    html_content = driver.page_source

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html_content, "html.parser")
    
    title = soup.select_one('#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.data_area > div.tit > strong')

    # 이름 가져오기
    try:
        title_name = title.get_text()
    except:
        return False
    cop = soup.select_one('#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.data_area > div.tit > p')

    # 회사 이름
    cop_name = cop.get_text()
    
    try:
        img = soup.select_one('#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.plate_area > div > img')
        # 이미지 주소 값 가져오기
        img_add = img['src']
    except:
        # 이미지 주소 크롤링 에러
        img_add = 'x'
    
    # 메인 정보 가져오기
    main_info_dls = soup.select('#q-app > section > div.card_detail.fr-view > section > div > article.card_top > div > div > div.data_area > div.bnf1 dl')

    main_info = []

    for dl in main_info_dls:
        main_info.append(dl.get_text())

    # 세부 정보 가져오기
    # dl 태그 가져오기
    dl_tags = soup.select('#q-app > section > div.card_detail.fr-view > section > div > article.cmd_con.benefit > div.lst.bene_area dl')

    details = []

    # 각 dl 태그에 대해 반복
    for dl_tag in dl_tags:
        # dt 태그와 dd 태그 데이터 가져오기
        dt_text = dl_tag.find('dt').get_text(strip=True)
        dd_text = dl_tag.find('dd').get_text(strip=True)

        details.append((dt_text, dd_text))
        
    card = {
        'index':i,
        'img':img_add,
        'cop':cop_name,
        'title':title_name,
        'main info':main_info,
        'details':details,}

    return card

In [361]:
cards = pd.DataFrame() # 카드 정보 데이터 프레임

In [362]:
# 3100

In [393]:

for i in range(2588, 3000):
    # 1. url 갱신
    url = f'https://www.card-gorilla.com/card/detail/{i}'
    
    # 2. 페이지 로딩
    driver.get(url)
    soup = load_page()
    
    # 3. 페이지 유효성 검사
    if check_valid(soup)==False:
        continue
    
    # 4. 카드 정보 가져오기
    card = get_card_info(i)
    if card ==False:
        continue
    
    # 5. 카드 정보 업데이트 해주기
    cards = pd.concat([cards, pd.DataFrame([card])], ignore_index=True)
    
    print(f'{i} 번째까지 완료')

2588 번째까지 완료
2589 번째까지 완료
2591 번째까지 완료
2592 번째까지 완료
2593 번째까지 완료
2594 번째까지 완료
2595 번째까지 완료
2596 번째까지 완료
2597 번째까지 완료
2598 번째까지 완료
2599 번째까지 완료
2600 번째까지 완료
2601 번째까지 완료
2602 번째까지 완료
2603 번째까지 완료
2604 번째까지 완료
2605 번째까지 완료
2606 번째까지 완료
2607 번째까지 완료
2608 번째까지 완료
2609 번째까지 완료
2611 번째까지 완료
2612 번째까지 완료
2613 번째까지 완료
2614 번째까지 완료
2615 번째까지 완료
2616 번째까지 완료
2617 번째까지 완료
2618 번째까지 완료
2619 번째까지 완료
2620 번째까지 완료
2621 번째까지 완료
2622 번째까지 완료
2625 번째까지 완료
2626 번째까지 완료
2627 번째까지 완료
2628 번째까지 완료
2629 번째까지 완료
2630 번째까지 완료
2631 번째까지 완료
2632 번째까지 완료
2633 번째까지 완료
2634 번째까지 완료
2635 번째까지 완료
2636 번째까지 완료


KeyboardInterrupt: 

In [394]:
cards

,index,img,cop,title,main info,details
0,8,https://api.card-gorilla.com:8080/storage/card...,신한카드,신한카드 The CLASSIC-Y,"[Gift Option매년1회선택이용, 마이신한포인트0.7~5%적립, GS칼텍스60...",[(선택형Gift Option Service - 매년 1회 옵션 품목 중 한 가지를...
1,11,https://api.card-gorilla.com:8080/storage/card...,신한카드,신한카드 Simple+,"[모든가맹점0.7%캐시백, 이동통신요금0.7%추가캐시백, 영화1,500원할인]","[(모든가맹점전가맹점 0.7% 캐시백이동통신요금 자동이체 건은 1.4% 캐시백, 전..."
2,15,https://api.card-gorilla.com:8080/storage/card...,신한카드,신한카드 The CLASSIC+,"[1,500원당1마일적립, 1,000원당1마일적립, PP카드무료제공]","[(바우처Gift Option, Gift Option 서비스는 매년 1회 아래 품목..."
3,21,https://api.card-gorilla.com:8080/storage/card...,신한카드,신한카드 Simple Platinum#,"[모든가맹점1%캐시백, 생활밀착가맹점0.7%추가캐시백, 영화3,000원할인]","[(모든가맹점전 가맹점 1%, 생활 밀착 가맹점 추가 0.7% 캐시백, 전 가맹점 ..."
4,28,https://api.card-gorilla.com:8080/storage/card...,신한카드,신세계 신한카드,"[신세계백화점5%할인, Life Care가맹점5%할인, 해외5%캐시백]","[(백화점신세계백화점 5% 결제일 할인, 신세계백화점 5% 결제일 할인- 전월 이용..."
...,...,...,...,...,...,...
1464,2632,https://api.card-gorilla.com:8080/storage/card...,롯데카드,디지로카 London,"[국내·해외최대 0.7%할인, 즉시결제최대 1%할인, 추가수수료없이나누어납부]","[(국내외가맹점어디서나 0.7% 캐시백, 어디서나 0.7% 캐시백- 국내 모든 가맹..."
1465,2633,https://api.card-gorilla.com:8080/storage/card...,롯데카드,디지로카 Paris,"[국내·해외0.7%할인, 온라인 쇼핑최대 5%할인]","[(국내외가맹점어디서나 0.7% 결제일 할인, 어디서나 0.7% 결제일 할인- 국내..."
1466,2634,https://api.card-gorilla.com:8080/storage/card...,롯데카드,디지로카 Monaco,"[국내·해외1%할인, 온라인쇼핑최대 5%할인, 배달앱/이동통신/스트리밍5~50%할인]","[(국내외가맹점어디서나 1% 결제일 할인, 어디서나 1% 결제일 할인국내 모든 가맹..."
1467,2635,https://api.card-gorilla.com:8080/storage/card...,롯데카드,Triple In LOCA,"[이동통신1만원할인, 렌탈1만원할인, 보험료1만원할인]","[(선택형Triple in 혜택5개의 생활 업종 중 3개 최대 30,000원 할인,..."


In [395]:
cards.to_csv("cards_sub.csv", index=False, encoding="utf-8-sig")